# Planning with multiple agents

In this sample we will be creating agents and use a tool (function) to provide answers to questions. We will also be creating a thread and then doing a follow-up with the agent to ask for more information.

You should have the following two environment variables already created and initialized:

* `AZURE_AI_PROJECT_ENDPOINT`: Your Azure AI project endpoint
* `AZURE_AI_MODEL_DEPLOYMENT_NAME`: The name of your model deployment

In [ ]:
from pydantic import BaseModel, Field
from typing import List
from agent_framework import ChatMessage, Role, ChatOptions
from agent_framework import ChatAgent, ai_function
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
import os

project_endpoint = os.environ.get('AZURE_AI_PROJECT_ENDPOINT')
model_name = os.environ.get('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print(f"Project endpoint: {project_endpoint}")
print(f"Deployment name: {model_name}")

In [ ]:
# 🎯 Agent Configuration
# Define the agent's name and role

AGENT_NAME = "TravelAgent"

# 📋 System Instructions for Planning Agent
# This agent acts as a coordinator that decides which specialized agents to use
AGENT_INSTRUCTIONS = """You are a planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialized in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
"""

In [ ]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [ ]:
# Create the AI Agent Client
client = AzureAIAgentClient(async_credential=AzureCliCredential())

In [ ]:
agent =  client.create_agent(name= AGENT_NAME , instructions=AGENT_INSTRUCTIONS)

In [ ]:
options = ChatOptions(response_format=TravelPlan)

In [ ]:
messages = [
        ChatMessage(role=Role.USER, text="Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne")
    ]

In [ ]:
result = await agent.run(messages, response_format=TravelPlan)

In [ ]:
print(result.text)

In [ ]:
client.close()